# Creazione ambiente great_expectations 

great_expectations puó essere installato direttamente da pip
(```pip install great-expectations```).

Per creare una suite di tests é necessario inizializzare un progetto facendo girare, da linea di comando,
```great_expectations init```

Questo comando crea una serie di cartelle nelle quali verranno salvati tutti i test creati e la documentazione HTML.



In [ ]:
import great_expectations as ge

data_context = ge.data_context.DataContext()
suite = data_context.create_expectation_suite("prova1",overwrite_existing=True,)

In [ ]:
suite

# Import di un batch di dati a partire da un pandas DataFrame

In [ ]:
batch = ge.dataset.PandasDataset(pd.read_csv('./X_exercise.csv'), expectation_suite=suite)

In [ ]:
batch

In [ ]:
# Etichetta target
batch['FraudFound_P'].value_counts()

# Creiamo una expectation per una singola colonna


In [ ]:
batch['WeekOfMonth'].unique()

In [ ]:
batch.expect_column_values_to_be_between('WeekOfMonth',1,5)

# Aggiungiamo altre expectations 

In [ ]:
batch.expect_column_max_to_be_between('Year',1994,2000)

In [ ]:
batch.expect_column_min_to_be_between('Age',16,20)
batch.expect_column_min_to_be_between('Year',1990,1994)

# Aggiungiamo un expectation con un elemento probabilistico


In [ ]:
batch.expect_column_values_to_be_between('Age',15,85, mostly=0.9)

# Aggiungiamo una expectation condizionale


In [ ]:
batch.expect_column_values_to_be_between(
                column='Age',
                min_value=25,
                max_value=80,
                condition_parser="pandas",
                row_condition='MaritalStatus=="Married"',
                mostly=0.95,
            )

# Salviamo le expectations definite fino ad ora all'interno della nostra suite

In [ ]:
suite = batch.get_expectation_suite()
data_context.save_expectation_suite(suite)

# Generiamo data_docs in formato HTML

In [ ]:
data_context.build_data_docs()

# Utilizziamo la suite creata per validare un nuovo batch di dati



In [ ]:
import datetime
batch_test = ge.dataset.PandasDataset(pd.read_csv('./X_holdout.csv'), expectation_suite=suite)
run_id = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%S.%fZ")
validation_result = batch_test.validate(run_id=run_id,catch_exceptions=True)

In [ ]:
validation_result

# Proviamo a far fallire qualche expectation

In [ ]:
X = pd.read_csv('./X_holdout.csv')
X['Age'].iloc[np.random.choice(X.shape[0],150)]=0
X.to_csv('./X_holdout_corrupted.csv')

In [ ]:
import datetime
batch_test = ge.dataset.PandasDataset(pd.read_csv('./X_holdout_corrupted.csv'), expectation_suite=suite)
run_id = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%S.%fZ")
validation_result = batch_test.validate(run_id=run_id,catch_exceptions=True)
validation_result

# Generazione automatica di data tests tramite libreria di data profiling

In [ ]:
from structured_data_profiling.profiler import DatasetProfiler

Istanziamo la classe DatasetProfiler specificando eventuali presenze di primary_key, colonne target e attributi protetti

In [ ]:
dp = DatasetProfiler('./X_exercise.csv', 
                     regression=False, 
                     primary_key='PolicyNumber',
                     target='FraudFound_P', 
                     protected_attributes=['Sex','Age'])

In [ ]:
dp.column_types

In [ ]:
dp.profile()

In [ ]:
dp.summary()

In [ ]:
dp.ordinal_columns

In [ ]:
dp.column_profiles['NumberOfSuppliments']


In [ ]:
dp.column_profiles['Make']

#str(dp.reduced_data_sample['Month'].iloc[10])


# Generiamo una suite di expectations a partire dai test costruiti

In [ ]:
dp.generate_expectations()

# Analizziamo i segmenti di dati identificati dal profiler 

In [ ]:
dp.data_slices

In [ ]:
# Analizziamo le data slices trovate dal profiler

for i in dp.data_slices:
    X_i = dp.data_sample.query(i)
    label_count = X_i['FraudFound_P'].value_counts().values
    fraction = 0
    if label_count.shape[0]>1:
        fraction = label_count[1]/label_count[0]
    print(i, ', Positive_label_fraction: ', fraction)    